# Sentinel-2 multiclass classification using Convolutional Neural Network

## Download the EuroSAT dataset


[EuroSAT dataset](https://github.com/phelber/EuroSAT)
![](https://raw.githubusercontent.com/phelber/EuroSAT/master/eurosat_overview_small.jpg)


Download the dataset

In [ ]:
import requests

url =  'http://madm.dfki.de/files/sentinel/EuroSAT.zip'
r = requests.get(url, allow_redirects=True)
open('EuroSAT.zip', 'wb').write(r.content)

Unzip the dataset

In [ ]:
!unzip '/content/EuroSAT.zip'

Rename the folder

In [ ]:
!mv '/content/2750' '/content/EuroSAT'

Count images in the dataset

In [ ]:
import glob

paths = glob.glob('/content/EuroSAT/*/*')
print('Number of images {}/27000'.format(len(paths)))

## Image Data Generator 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/EuroSAT',  
    target_size=(64, 64),  
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    '/content/EuroSAT',  
    target_size=(64, 64),  
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  

In [ ]:
classes = train_generator.class_indices
classes= dict(zip(classes.values(), classes.keys()))  
print(classes)

Generate some exaples

In [ ]:
x,y = next(iter(train_generator))
print('x shape {}'.format(x.shape))
print('y shape {}'.format(y.shape))

Plot some examples

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize = (16,16))

ct = 0
for i in range(4):
  for j in range(4):
    axes[i,j].imshow(x[ct])
    axes[i,j].set_title('Label {} \n Class {}'.format(y[ct], classes[np.argmax(y[ct])]))
    axes[i,j].axis(False)
    ct += 1

fig.tight_layout()
plt.show()


## Define the Convolutional Neural Network

We will use the LeNet-5 ([Find out more](https://en.wikipedia.org/wiki/LeNet))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Input, Dropout


model = None
x_in = Input(shape = (64,64,3))


# Convolution
x = Conv2D(filters=6, kernel_size=(5, 5), activation='relu')(x_in)
x = AveragePooling2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(filters=16, kernel_size=(5, 5), activation='relu')(x)
x = AveragePooling2D(pool_size=(2,2), strides=(2,2))(x)
# MLP
x = Flatten()(x)
x = Dense(units=120, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(units=84, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(units=10, activation = 'softmax')(x)

model = Model(inputs = x_in, outputs = x)

Compile the model

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

Print model configuration

In [ ]:
model.summary()

## Train the LeNet-5

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.n//batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.n//batch_size,
    epochs = 15
)

Plot training history

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (10,5))

axes[0].plot(history.history['loss'], '-*', label = 'Training')
axes[0].plot(history.history['val_loss'], '-o', label = 'Validtion')
axes[0].set_ylabel('Categorical Cross Entropy')
axes[0].set_xlabel('Epochs')
axes[0].set_title('Model loss over epochs')

axes[1].plot(history.history['accuracy'], '-*', label = 'Training')
axes[1].plot(history.history['val_accuracy'], '-o', label = 'Validtion')
axes[1].set_ylabel('Accuracy')
axes[1].set_xlabel('Epochs')
axes[1].set_title('Model accuracy over epochs')

fig.tight_layout()
plt.show()

## Evaluate Performances

In [ ]:
batch_size = val_generator.n

val_generator = train_datagen.flow_from_directory(
    '/content/EuroSAT',  
    target_size=(64, 64),  
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') 

Get predictions for the validation set

In [ ]:
x, y = next(iter(val_generator))
y_true = np.argmax(y, axis=1)
Y_pred = model.predict(x)
y_pred = np.argmax(Y_pred, axis=1)

Compute confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred, normalize='true')

Plot confusion matrix

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes.values())
disp.plot(ax=ax, cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.show()

Compute classification report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=classes.values()))